In [13]:
import pandas as pd
import selenium
from selenium import webdriver
import time
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
from time import sleep 
from selenium.webdriver.support.wait import WebDriverWait as wait
import numpy as np

import warnings 
warnings.filterwarnings('ignore')
import urllib
import os
from bs4 import BeautifulSoup
from selenium.webdriver.support import expected_conditions as EC

In [14]:
driver = webdriver.Firefox()

In [15]:
df_main=pd.DataFrame(columns=["Name","Duration","Layovers","Departure_time",
                      "Arrival_time","Source","Destination","Details","Price"])

In [16]:
df_main

,Name,Duration,Layovers,Departure_time,Arrival_time,Source,Destination,Details,Price


In [26]:
def data_mine(link):

    driver.get(link)
    wait(driver, 5).until(EC.element_to_be_clickable((By.XPATH,"//div[@class='link-color fs-12 cursor-pointer']")))
       
    #infinite scrolling
    SCROLL_PAUSE_TIME = 1

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    #name of the airline
    nametags=driver.find_elements_by_xpath("//span[@class='i-b text ellipsis']")
    name=[]
    for n in nametags:
        try:
            name.append(n.text)
        except NoSuchElementException:
            name.append(np.nan)
#     sleep(1)
    #duration of the journey
    durtags = driver.find_elements_by_xpath("//p[@class='fs-12 bold du mb-2']")
    dur=[]
    for d in durtags:
        try:
            dur.append(d.text)
        except NoSuchElementException:
            dur.append(np.nan)
#     sleep(1)
    #emission and meal availability
    footertags = driver.find_elements_by_xpath("//div[@class='features pull-right fs-12 flex']")
    footer=[]
    for f in footertags:
        try:
            footer.append(f.text)
        except NoSuchElementException:
            footer.append("Not available")
#     sleep(1)
    #price of the journey
    pricetags = driver.find_elements_by_xpath("//div[@class='i-b tipsy fare-summary-tooltip fs-18']")
    price=[]
    for p in pricetags:
        try:
            price.append(p.text)
        except NoSuchElementException:
            price.append(np.nan)
#     sleep(1)
    #arrival time
    atimetags = driver.find_elements_by_xpath("//div[@class='i-b pdd-0 text-left atime col-5']/p[1]")
    arrtime=[]
    for a in atimetags:
            arrtime.append(a.text)
#     sleep(1)
    #departure time        
    dtimetags = driver.find_elements_by_xpath("//div[@class='i-b pr']")
    deptime=[]
    for d in dtimetags:
            deptime.append(d.text)
            
#     sleep(1)        
    #number of stops
    stoptags = driver.find_elements_by_xpath("//div[@class=' font-lightgrey fs-10 tipsy i-b fs-10']")
    stops=[]
    for s in stoptags:
            stops.append(s.text)

#     sleep(1)
    #Origin city
    sourcetags = driver.find_elements_by_xpath("//div[@class='i-b col-4 no-wrap text-right dtime col-3']/p[1]")
    source=[]
    for so in sourcetags:
            source.append(so.text) 
    
#     sleep(1)
    #destination city        
    desttags = driver.find_elements_by_xpath("//div[@class='i-b pdd-0 text-left atime col-5']/p[2]")
    dest=[]
    for de in desttags:
            dest.append(de.text)

    df_temp=pd.DataFrame({"Name":name,"Duration":dur,"Layovers":stops,"Departure_time":deptime,
                          "Arrival_time":arrtime,"Source":source,"Destination":dest,
                          "Details":footer,"Price":price})
    

    return df_temp

In [27]:
#Del to Mumbai
temp1=data_mine("https://flight.yatra.com/air-search-ui/dom2/trigger?ADT=1&CHD=0&INF=0&class=Economy&destination=BOM&destinationCountry=IN&flexi=0&flight_depart_date=01%2F03%2F2022&hb=0&noOfSegments=1&origin=DEL&originCountry=IN&type=O&unique=90109695219&viewName=normal")
temp1.shape

(142, 9)

In [28]:
#Mumbai to Del
temp2=data_mine("https://flight.yatra.com/air-search-ui/dom2/trigger?type=O&viewName=normal&flexi=0&noOfSegments=1&origin=BOM&originCountry=IN&destination=DEL&destinationCountry=IN&flight_depart_date=01%2F03%2F2022&ADT=1&CHD=0&INF=0&class=Economy&source=fresco-home&unqvaldesktop=1049077088347")
temp2.shape

(162, 9)

In [29]:
#beng to hyder
temp3=data_mine("https://flight.yatra.com/air-search-ui/dom2/trigger?ADT=1&CHD=0&INF=0&class=Economy&destination=HYD&destinationCountry=IN&flexi=0&flight_depart_date=01%2F03%2F2022&hb=0&noOfSegments=1&origin=BLR&originCountry=IN&type=O&unique=833829049588&version=1.1&viewName=normal")
temp3.shape

(120, 9)

In [33]:
#hyder to beng
temp4=data_mine("https://flight.yatra.com/air-search-ui/dom2/trigger?ADT=1&CHD=0&INF=0&class=Economy&destination=BLR&destinationCountry=IN&flexi=0&flight_depart_date=01%2F03%2F2022&hb=0&noOfSegments=1&origin=HYD&originCountry=IN&type=O&unique=514890830813&version=1.1&viewName=normal")
temp4.shape

(104, 9)

In [34]:
#pune to chennai
temp17 = data_mine("https://flight.yatra.com/air-search-ui/dom2/trigger?ADT=1&CHD=0&INF=0&class=Economy&destination=MAA&destinationCountry=IN&flexi=0&flight_depart_date=01%2F03%2F2022&hb=0&noOfSegments=1&origin=PNQ&originCountry=IN&type=O&unique=602166529091&version=1.1&viewName=normal")
temp17.shape

(73, 9)

In [36]:
#chennai to pune
temp5 = data_mine("https://flight.yatra.com/air-search-ui/dom2/trigger?ADT=1&CHD=0&INF=0&class=Economy&destination=PNQ&destinationCountry=IN&flexi=0&flight_depart_date=01%2F03%2F2022&hb=0&noOfSegments=1&origin=MAA&originCountry=IN&type=O&unique=1493964305913&version=1.1&viewName=normal")
temp5.shape

(78, 9)

In [37]:
#kolkata to jaipur
temp6=data_mine("https://flight.yatra.com/air-search-ui/dom2/trigger?ADT=1&CHD=0&INF=0&class=Economy&destination=JAI&destinationCountry=IN&flexi=0&flight_depart_date=01%2F03%2F2022&hb=0&noOfSegments=1&origin=CCU&originCountry=IN&type=O&unique=974022677252&version=1.1&viewName=normal")
temp6.shape

(84, 9)

In [39]:
#jaipur to kol
temp7=data_mine("https://flight.yatra.com/air-search-ui/dom2/trigger?ADT=1&CHD=0&INF=0&class=Economy&destination=CCU&destinationCountry=IN&flexi=0&flight_depart_date=01%2F03%2F2022&hb=0&noOfSegments=1&origin=JAI&originCountry=IN&type=O&unique=1080523056542&version=1.1&viewName=normal")
temp7.shape

(87, 9)

In [40]:
#mumbai to kol
temp8=data_mine("https://flight.yatra.com/air-search-ui/dom2/trigger?ADT=1&CHD=0&INF=0&class=Economy&destination=CCU&destinationCountry=IN&flexi=0&flight_depart_date=01%2F03%2F2022&hb=0&noOfSegments=1&origin=BOM&originCountry=IN&type=O&unique=1553310394620&version=1.1&viewName=normal")
temp8.shape

(146, 9)

In [42]:
#kol to mum
temp9=data_mine("https://flight.yatra.com/air-search-ui/dom2/trigger?ADT=1&CHD=0&INF=0&class=Economy&destination=BOM&destinationCountry=IN&flexi=0&flight_depart_date=01%2F03%2F2022&hb=0&noOfSegments=1&origin=CCU&originCountry=IN&type=O&unique=1055811029024&version=1.1&viewName=normal")
temp9.shape

(141, 9)

In [43]:
#del to chen
temp10=data_mine("https://flight.yatra.com/air-search-ui/dom2/trigger?ADT=1&CHD=0&INF=0&class=Economy&destination=MAA&destinationCountry=IN&flexi=0&flight_depart_date=01%2F03%2F2022&hb=0&noOfSegments=1&origin=DEL&originCountry=IN&type=O&unique=1383308732115&version=1.1&viewName=normal")
temp10.shape

(152, 9)

In [45]:
#chen to del
temp11=data_mine("https://flight.yatra.com/air-search-ui/dom2/trigger?ADT=1&CHD=0&INF=0&class=Economy&destination=DEL&destinationCountry=IN&flexi=0&flight_depart_date=01%2F03%2F2022&hb=0&noOfSegments=1&origin=MAA&originCountry=IN&type=O&unique=999734043387&version=1.1&viewName=normal")
temp11.shape

(144, 9)

In [46]:
#ahmedabad to ben
temp12=data_mine("https://flight.yatra.com/air-search-ui/dom2/trigger?ADT=1&CHD=0&INF=0&class=Economy&destination=BLR&destinationCountry=IN&flexi=0&flight_depart_date=01%2F03%2F2022&hb=0&noOfSegments=1&origin=AMD&originCountry=IN&type=O&unique=43940924505&version=1.1&viewName=normal")
temp12.shape

(109, 9)

In [48]:
#beng to ahm
temp13=data_mine("https://flight.yatra.com/air-search-ui/dom2/trigger?type=O&viewName=normal&flexi=0&noOfSegments=1&origin=BLR&originCountry=IN&destination=AMD&destinationCountry=IN&flight_depart_date=01%2F03%2F2022&ADT=1&CHD=0&INF=0&class=Economy&source=fresco-home&unqvaldesktop=1081641336008")
temp13.shape

(107, 9)

In [49]:
#goa to del
temp14=data_mine("https://flight.yatra.com/air-search-ui/dom2/trigger?ADT=1&CHD=0&INF=0&class=Economy&destination=DEL&destinationCountry=IN&flexi=0&flight_depart_date=01%2F03%2F2022&hb=0&noOfSegments=1&origin=GOI&originCountry=IN&type=O&unique=1145275887780&version=1.1&viewName=normal")
temp14.shape

(140, 9)

In [51]:
#del to goa
temp15=data_mine("https://flight.yatra.com/air-search-ui/dom2/trigger?ADT=1&CHD=0&INF=0&class=Economy&destination=GOI&destinationCountry=IN&flexi=0&flight_depart_date=01%2F03%2F2022&hb=0&noOfSegments=1&origin=DEL&originCountry=IN&type=O&unique=1240899546952&version=1.1&viewName=normal")
temp15.shape

(158, 9)

In [52]:
#mum to goa
temp18=data_mine("https://flight.yatra.com/air-search-ui/dom2/trigger?ADT=1&CHD=0&INF=0&class=Economy&destination=GOI&destinationCountry=IN&flexi=0&flight_depart_date=01%2F03%2F2022&hb=0&noOfSegments=1&origin=BOM&originCountry=IN&type=O&unique=798285892997&version=1.1&viewName=normal")
temp18.shape

(102, 9)

In [53]:
#goa to mum
temp16=data_mine("https://flight.yatra.com/air-search-ui/dom2/trigger?ADT=1&CHD=0&INF=0&class=Economy&destination=BOM&destinationCountry=IN&flexi=0&flight_depart_date=01%2F03%2F2022&hb=0&noOfSegments=1&origin=GOI&originCountry=IN&type=O&unique=1567004015738&version=1.1&viewName=normal")
temp16.shape

(90, 9)

In [54]:
df_main=pd.concat([temp1,temp2,temp3,temp4,temp5,temp6,temp7,
                   temp8,temp9,temp10,temp11,temp12,temp13,temp14,
                   temp15,temp16,temp17,temp18],ignore_index=True)

In [55]:
df_main

,Name,Duration,Layovers,Departure_time,Arrival_time,Source,Destination,Details,Price
0,Air Asia,5h 50m,1 Stop,08:20,14:10,New Delhi,Mumbai,eCash 250,"5,949"
1,Air Asia,6h 25m,1 Stop,20:00,02:25\n+ 1 day,New Delhi,Mumbai,eCash 250,"5,949"
2,Air Asia,10h 30m,1 Stop,20:45,07:15\n+ 1 day,New Delhi,Mumbai,Emissions: 303 Kg CO2\neCash 250,"5,949"
3,Air Asia,11h 00m,1 Stop,09:35,20:35,New Delhi,Mumbai,Emissions: 303 Kg CO2\neCash 250,"5,949"
4,Air Asia,11h 40m,1 Stop,20:00,07:40\n+ 1 day,New Delhi,Mumbai,eCash 250,"5,949"
...,...,...,...,...,...,...,...,...,...
2134,Vistara,18h 05m,1 Stop,19:45,13:50\n+ 1 day,Mumbai,Goa,No Meal Fare\nEmissions: 565 Kg CO2\neCash 250,"25,893"
2135,Vistara,19h 20m,1 Stop,18:30,13:50\n+ 1 day,Mumbai,Goa,No Meal Fare\nEmissions: 565 Kg CO2\neCash 250,"25,893"
2136,Vistara,20h 15m,1 Stop,17:35,13:50\n+ 1 day,Mumbai,Goa,No Meal Fare\nEmissions: 565 Kg CO2\neCash 250,"25,893"
2137,Vistara,22h 05m,1 Stop,15:45,13:50\n+ 1 day,Mumbai,Goa,No Meal Fare\nEmissions: 565 Kg CO2\neCash 250,"25,893"


In [57]:
df_main.to_csv("Fligh_prices_dataset.csv")